In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w3-p2/train_data.csv
/kaggle/input/2021-ai-w3-p2/test_data.csv
/kaggle/input/2021-ai-w3-p2/sample_submit.csv


In [2]:
import torch
import torch.optim as optim

torch.manual_seed(1)

In [3]:
X = pd.read_csv('/kaggle/input/2021-ai-w3-p2/train_data.csv')
y = X['price']
X = X.drop(['ID', 'price', 'model', 'year', 'transmission', 'fueltype'], axis=1)
test = pd.read_csv('/kaggle/input/2021-ai-w3-p2/test_data.csv').drop(['ID' ,'model', 'year', 'transmission', 'fueltype'], axis=1)
sample = pd.read_csv('/kaggle/input/2021-ai-w3-p2/sample_submit.csv')

In [4]:
X.describe()

,company,mileage,tax,mpg,engineSize
count,17085.000000,17085.000000,17085.000000,17085.000000,17085.000000
mean,1.955926,23811.395142,118.696225,56.449874,1.671659
std,1.294944,21403.239323,65.087240,18.745561,0.599490
min,0.000000,1.000000,0.000000,2.800000,0.000000
25%,1.000000,7881.000000,125.000000,47.900000,1.200000
50%,2.000000,18266.000000,145.000000,56.500000,1.600000
75%,3.000000,33322.000000,145.000000,64.200000,2.000000
max,4.000000,178987.000000,580.000000,470.800000,6.300000


In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
X['tax'] = scaler.fit_transform(X['tax'].to_numpy().reshape(-1, 1))
test['tax']= scaler.transform(test['tax'].to_numpy().reshape(-1, 1))
X['mileage'] = scaler.fit_transform(X['mileage'].to_numpy().reshape(-1, 1))
test['mileage']= scaler.transform(test['mileage'].to_numpy().reshape(-1, 1))
X['mpg'] = scaler.fit_transform(X['mpg'].to_numpy().reshape(-1, 1))
test['mpg']= scaler.transform(test['mpg'].to_numpy().reshape(-1, 1))

In [6]:
X = torch.FloatTensor(X.to_numpy())
y = torch.FloatTensor(y.to_numpy())
test = torch.FloatTensor(test.to_numpy())

In [7]:
W = torch.zeros((5, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1e-2)
nb_epochs = 600
for epoch in range(nb_epochs + 1):
    
    hypothesis = X.matmul(W) + b 
    # cost 계산
    cost = torch.mean((hypothesis - y) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if(epoch%10==0):
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), math.sqrt(cost.item())
        ))
        

Epoch    0/600 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 19575.600323
Epoch   10/600 hypothesis: tensor([14899.7637,  7933.2178, 17018.3730,  ..., 18564.7949, 11022.8574,
        12466.3184]) Cost: 11436.231897
Epoch   20/600 hypothesis: tensor([17493.0410,  9388.5215, 19506.4453,  ..., 20801.8164, 12623.6484,
        14435.5010]) Cost: 10991.943231
Epoch   30/600 hypothesis: tensor([18136.1172,  9817.4727, 19790.5020,  ..., 20660.4688, 12810.6016,
        14790.9707]) Cost: 10857.041218
Epoch   40/600 hypothesis: tensor([18425.7188, 10060.3564, 19751.0977,  ..., 20249.2402, 12802.2598,
        14892.2236]) Cost: 10768.503703
Epoch   50/600 hypothesis: tensor([18617.0430, 10251.2881, 19673.0312,  ..., 19871.6621, 12780.5732,
        14948.5078]) Cost: 10706.472061
Epoch   60/600 hypothesis: tensor([18758.0391, 10415.3301, 19597.7461,  ..., 19559.1602, 12769.9307,
        14992.9922]) Cost: 10661.604757
Epoch   70/600 hypothesis: tensor([18863.1738, 10559.9551, 19529.9082

In [8]:
predict = test.matmul(W) + b 
predict = predict.detach().numpy()

In [9]:
sample['price'] = predict
sample.head()

,ID,price
0,0,15175.324219
1,1,17519.197266
2,2,18352.109375
3,3,13778.089844
4,4,15824.626953


In [10]:
sample.to_csv('submit.csv', index = False)